# BlueSky Flyer that uses the synApps busy record

EPICS fly scans that use external controllers, such as hardware-assisted fly scans, are triggered by the EPICS *busy* record.  The *busy* record is set, which triggers the external controller to do the fly scan and then reset the *busy* record.

Here, we refer to this external controller as the *controller*.


## Python imports and definitions <a class="anchor" id="imports" />

Here are the full set of packages to imported.  The first block are Python standard packages, then come the ophyd and BluSky packages.  Just the parts we plan on using here.  Since this is also a tutorial, we will not rename imports or use other such shortcuts in the documentation (the online code has some shortcuts).

* Create a logger instance in case we want to investigate internal details as our code runs.
* Create an instance of the BlueSky RunEngine.
* Create an instance of the databroker using the `mongodb_config.yml` file on the local machine
* Arrange for the databroker to receive all events from the RunEngine


In [1]:
import enum
import logging
import threading
import time

import ophyd
import bluesky
import bluesky.plans
import databroker

logger = logging.getLogger()
RE = bluesky.RunEngine({})
db = databroker.Broker.named("mongodb_config")
RE.subscribe(db.insert)

0

## EPICS PV names <a class="anchor" id="pvnames" />

These are the EPICS PV names we'll be using for the demo, as defined in this EPICS database, added to our EPICS IOC startup:

```
record(waveform, "$(P)str_wave")
{
    field(FTVL, "CHAR")
    field(NELM, "256")
}
record(waveform, "$(P)t_array")
{
    field(DESC, "timestamps")
    field(FTVL, "DOUBLE")
    field(NELM, "256")
}
record(waveform, "$(P)x_array")
{
    field(DESC, "positions")
    field(FTVL, "DOUBLE")
    field(NELM, "256")
}
record(waveform, "$(P)y_array")
{
    field(DESC, "signals")
    field(FTVL, "DOUBLE")
    field(NELM, "256")
}
```

The waveforms allow for up to 256 values to be kept.

In [2]:
BUSY_PV = 'prj:mybusy'
TIME_WAVE_PV = 'prj:t_array'
X_WAVE_PV = 'prj:x_array'
Y_WAVE_PV = 'prj:y_array'

## External pseudo-controller for fly scan <a class="anchor" id="pseudo-controller" />

We'll create a *controller* that responds to the EPICS PV that signals a fly scan should be started.  The *controller* operates the fly scan and stores the collected data in EPICS PVs, then signals EPICS that the fly scan is complete.  BlueSky will observe that the fly scan is complete and collect the data from EPICS.

A Python program has been built to act as the *controller*.  This *controller* moves a soft [motor](https://github.com/epics-modules/motor) record in a step scan and reads from an [swait](https://github.com/epics-modules/calc) record that has been configured to generate random numbers.

The [pseudo-controller](https://github.com/prjemian/ipython_mintvm/blob/master/profile_bluesky/startup/local_code/busyExample.py) *and associated EPICS IOC* should be started outside of this jupyter notebook.  Since the interface is completely through EPICS PVs, it is not necessary for either the IOC or the pseudo-controller to be running on the some computer as the jupyter notebook.

![control screen for the motor record](motor.png "control screen for the motor record")

![control screen for the swait record](swait.png "control screen for the swait record")

## Devices and support <a class="anchor" id="devices" />

Define some ophyd Devices we'll use and support routines for them.

### synApps busy record <a class="anchor" id="busy-record" />

The synApps [busy](https://github.com/epics-modules/busy) record is a variation of a 0,1 (bit).  It is an integer that starts at zero.  *Busy* is set to one at the start of operations.  This triggers execution of one or more operations that must complete before *busy* is set back to zero.  

![control screen for the busy record](busy.png "control screen for the busy record")

In [3]:
class BusyRecord(ophyd.Device):
    """a busy record sets the fly scan into action"""
    state = ophyd.Component(ophyd.EpicsSignal, "", string=True)
    output_link = ophyd.Component(ophyd.EpicsSignal, ".OUT")
    forward_link = ophyd.Component(ophyd.EpicsSignal, ".FLNK")

We won't need to use the `.OUT` or `.FLNK` fields in this example.


### busy status values <a class="anchor" id="busy-status-values" />

It is useful to define the state values (strings) so that clients have no ambiguity in spelling and error checking.

use this | instead of
---- | ----
`BusyStatus.busy` | `"Busy"`
`BusyStatus.done` | `"Done"`

In [4]:
class BusyStatus(str, enum.Enum):
    busy = "Busy"
    done = "Done"

We want a couple items from the [waveform](https://wiki-ext.aps.anl.gov/epics/index.php/RRM_3-14_Waveform) record.  Let's use a custom ophyd Device for just what we need here.

In [5]:
class MyWaveform(ophyd.Device):
    """waveform records store fly scan data"""
    wave = ophyd.Component(ophyd.EpicsSignalRO, "")
    number_elements = ophyd.Component(ophyd.EpicsSignalRO, ".NELM")
    number_read = ophyd.Component(ophyd.EpicsSignalRO, ".NORD")

## Final, working Flyer <a class="anchor" id="working-flyer" />

We'll start from the **Flyer that "collects" 1-D array data** as described in the `flyer_template` notebook (same directory as this notebook).

If we want to poll the busy PV for it to be Done, then we needn't tie up the CPU completely.  We can poll at a more limited rate by adding a delay time between polling cehcks of the busy state.  50 ms should be fine for a scan that involves moving a motor.  Add this to the constructor:

        self.poll_sleep_interval_s = 0.05

Later, this is used to wait for the busy record:

        while self.busy.state.value not in (BusyStatus.done):
            time.sleep(self.poll_sleep_interval_s)

Already, we added a starting time (`self.t0`) that is set at `kickoff()`.  This is used to measure elapsed time to each data reporting event.

When talking to EPICS PVs, we create instances of each and add them to our Device:

    busy = ophyd.Component(BusyRecord, BUSY_PV)
    tArr = ophyd.Component(MyWaveform, TIME_WAVE_PV)
    xArr = ophyd.Component(MyWaveform, X_WAVE_PV)
    yArr = ophyd.Component(MyWaveform, Y_WAVE_PV)

where the PV names (`BUSY_PV`, ...) are configured earlier in this notebook (where it can be seen easily by users).

The *activity* consists of making sure the busy record starts at `Done` before we try to fly scan.  Wait for that, just in case.  

    self.busy.state.put(BusyStatus.done)
    self.wait_busy() 

Then, set it's state to `Busy` and wait for it to return to `Done`.  

    self.t0 = time.time()
    self.busy.state.put(BusyStatus.busy)
    self.wait_busy() 

Once done, set the status object and return, ending the thread.

    self._completion_status._finished(success=True)

With real data, we need to modify both `collect()` and `describe_collect()` for each data to be yielded.  The names must match in both methods or the RunEngine will raise `KeyError: frozenset ...` and tell you about the data you tried to offer.  The names ***must match*** in both methods.  In the `collect()` method, don't forget to return the status object to its value of `None`, signifying that we are not flying.

See GitHub for a [summary of changes in source code](https://github.com/prjemian/ipython_mintvm/compare/a0af3ec57a3430e777b3...ce116e5e05774).

In [6]:
class MyFlyer(ophyd.Device):
    """
    a basic Flyer for scans triggered by the synApps busy record
    """

    busy = ophyd.Component(BusyRecord, BUSY_PV)
    tArr = ophyd.Component(MyWaveform, TIME_WAVE_PV)
    xArr = ophyd.Component(MyWaveform, X_WAVE_PV)
    yArr = ophyd.Component(MyWaveform, Y_WAVE_PV)

    def __init__(self, *args, **kwargs):
        super().__init__('', parent=None, **kwargs)
        self._completion_status = None
        self.poll_sleep_interval_s = 0.05
        self.t0 = 0

    def wait_busy(self, target = None):
        """
        wait for the busy record to return to the target value
        """
        logger.debug("wait_busy()")
        target = target or BusyStatus.done

        while self.busy.state.value not in (target):
            time.sleep(self.poll_sleep_interval_s)  # wait to complete ...
 
    def my_activity(self):
        """
        start the "fly scan" here, could wait for completion
        
        It's OK to use blocking calls here 
        since this is called in a separate thread
        from the BlueSky RunEngine.
        """
        logger.info("activity()")
        if self._completion_status is None:
            logger.info("leaving activity() - not complete")
            return
        
        # do the activity here
        self.busy.state.put(BusyStatus.done) # make sure it's Done first
        self.wait_busy()

        # wait for completion
        self.t0 = time.time()
        self.busy.state.put(BusyStatus.busy)
        self.wait_busy()
        
        self._completion_status._finished(success=True)
        logger.info("activity() complete. status = " + str(self._completion_status))

    def kickoff(self):
        """
        Start this Flyer
        """
        logger.info("kickoff()")
        self._completion_status = ophyd.DeviceStatus(self)
        
        thread = threading.Thread(target=self.my_activity, daemon=True)
        thread.start()

        kickoff_status = ophyd.DeviceStatus(self)
        kickoff_status._finished(success=True)
        return kickoff_status

    def complete(self):
        """
        Wait for flying to be complete
        """
        logger.info("complete()")
        if self._completion_status is None:
            raise RuntimeError("No collection in progress")

        return self._completion_status

    def describe_collect(self):
        """
        Describe details for ``collect()`` method
        """
        logger.info("describe_collect()")
        return {
            self.name: dict(
                ifly_xArr = dict(
                    source = self.xArr.wave.pvname,
                    dtype = "number",
                    shape = (1,)
                ),
                ifly_yArr = dict(
                    source = self.yArr.wave.pvname,
                    dtype = "number",
                    shape = (1,)
                ),
                ifly_tArr = dict(
                    source = self.tArr.wave.pvname,
                    dtype = "number",
                    shape = (1,)
                )
            )
        }

    def collect(self):
        """
        Start this Flyer
        """
        logger.info("collect()")
        self._completion_status = None
        for i in range(len(ifly.tArr.wave.value)):
            t = ifly.tArr.wave.value[i]
            x = ifly.xArr.wave.value[i]
            y = ifly.yArr.wave.value[i]
            d = dict(
                time=time.time(),
                data=dict(
                    ifly_tArr = time.time() - self.t0,
                    ifly_xArr = x,
                    ifly_yArr = y,
                ),
                timestamps=dict(
                    ifly_tArr = t,
                    ifly_xArr = t,
                    ifly_yArr = t,
                )
            )
            yield d


In [7]:
ifly = MyFlyer("prj:", name="ifly")

Verify that we connected with the busy record, *et al.* by printing the current state.

In [8]:
print(ifly.busy.state.pvname, ifly.busy.state.value)

prj:mybusy Done


In [9]:
ifly.describe_collect()

{'ifly': {'ifly_tArr': {'dtype': 'number',
   'shape': (1,),
   'source': 'prj:t_array'},
  'ifly_xArr': {'dtype': 'number', 'shape': (1,), 'source': 'prj:x_array'},
  'ifly_yArr': {'dtype': 'number', 'shape': (1,), 'source': 'prj:y_array'}}}

In [10]:
list(ifly.collect())

[{'data': {'ifly_tArr': 1524800743.0476582,
   'ifly_xArr': -1.23,
   'ifly_yArr': 0.64252689402609298},
  'time': 1524800743.047658,
  'timestamps': {'ifly_tArr': 1524800699.2301791,
   'ifly_xArr': 1524800699.2301791,
   'ifly_yArr': 1524800699.2301791}},
 {'data': {'ifly_tArr': 1524800743.0491278,
   'ifly_xArr': 0.87,
   'ifly_yArr': 0.48380254825665675},
  'time': 1524800743.0491276,
  'timestamps': {'ifly_tArr': 1524800701.6366019,
   'ifly_xArr': 1524800701.6366019,
   'ifly_yArr': 1524800701.6366019}},
 {'data': {'ifly_tArr': 1524800743.0521252,
   'ifly_xArr': 2.9700000000000002,
   'ifly_yArr': 0.85040054932478826},
  'time': 1524800743.052125,
  'timestamps': {'ifly_tArr': 1524800704.0421901,
   'ifly_xArr': 1524800704.0421901,
   'ifly_yArr': 1524800704.0421901}},
 {'data': {'ifly_tArr': 1524800743.0535603,
   'ifly_xArr': 5.0700000000000003,
   'ifly_yArr': 0.83656061646448465},
  'time': 1524800743.05356,
  'timestamps': {'ifly_tArr': 1524800706.448878,
   'ifly_xArr': 15

In [11]:
RE(bluesky.plans.fly([ifly]), md=dict(purpose="develop Flyer for APS fly scans"))

('249e9739-dd9d-4cce-afb3-f883c0c5434f',)

In [12]:
h = db[-1]
h.table(h.stream_names[0])

,time,ifly_xArr,ifly_yArr,ifly_tArr
seq_num,,,,
1,2018-04-26 22:46:01.325494,-1.23,0.042573,18.209701
2,2018-04-26 22:46:01.329039,0.87,0.455314,18.213246
3,2018-04-26 22:46:01.329623,2.97,0.178042,18.213829
4,2018-04-26 22:46:01.330728,5.07,0.127031,18.214934
5,2018-04-26 22:46:01.331759,7.17,0.928511,18.215966


In [13]:
list(h.documents())

[('start',
  {'md': {'purpose': 'develop Flyer for APS fly scans'},
   'plan_name': 'fly',
   'plan_type': 'generator',
   'scan_id': 1,
   'time': 1524800743.0884168,
   'uid': '249e9739-dd9d-4cce-afb3-f883c0c5434f'}),
 ('descriptor',
  {'data_keys': {'ifly_tArr': {'dtype': 'number',
     'shape': [1],
     'source': 'prj:t_array'},
    'ifly_xArr': {'dtype': 'number', 'shape': [1], 'source': 'prj:x_array'},
    'ifly_yArr': {'dtype': 'number', 'shape': [1], 'source': 'prj:y_array'}},
   'hints': {},
   'name': 'ifly',
   'object_keys': {'ifly': ['ifly_xArr', 'ifly_yArr', 'ifly_tArr']},
   'run_start': '249e9739-dd9d-4cce-afb3-f883c0c5434f',
   'time': 1524800761.3027973,
   'uid': 'f1844247-8beb-4535-abce-81d3436ba8c6'}),
 ('event',
  {'data': {'ifly_tArr': 18.2097008228302,
    'ifly_xArr': -1.23,
    'ifly_yArr': 0.04257267109178302},
   'descriptor': 'f1844247-8beb-4535-abce-81d3436ba8c6',
   'filled': {},
   'seq_num': 1,
   'time': 1524800761.325494,
   'timestamps': {'ifly_tArr